In [1]:
!pip install pdfplumber
!pip install spacy
!pip install sentence-transformers
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/

In [1]:
import pdfplumber
import spacy
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import io
from google.colab import files


In [3]:
# Dummy Resume Text (Normally from PDF)
resume_text = """
Ananta Kalaskar
Email: ananta@example.com
Skills: Python, Machine Learning, NLP, Pandas, Numpy, Scikit-learn, Deep Learning
Projects: EV Load Prediction, Product Review Sentiment Analysis, Resume Analyzer
Experience: Intern at AI Labs, Built ML models for real-world use
Education: B.Tech in Artificial Intelligence
"""

# Dummy Job Description Text (Normally pasted/uploaded)
jd_text = """
We are looking for a Machine Learning Engineer with:
- Strong skills in Python and Scikit-learn
- Experience in NLP and deep learning techniques
- Familiarity with Pandas, data preprocessing, and model deployment
- Bonus: Work on resume parsing or chatbot projects
"""


In [5]:
nlp = spacy.load("en_core_web_sm")

def extract_keywords(text, top_n=20):
    doc = nlp(text)
    keywords = [chunk.text.lower() for chunk in doc.noun_chunks if 1 < len(chunk.text.split()) < 4]
    return list(set(keywords))[:top_n]

resume_keywords = extract_keywords(resume_text)
jd_keywords = extract_keywords(jd_text)

print("✅ Resume Keywords:", resume_keywords)
print("📌 JD Keywords:", jd_keywords)


✅ Resume Keywords: ['machine learning', 'ev load prediction', 'resume analyzer\nexperience', 'ai labs', 'artificial intelligence', 'built ml models', 'real-world use\neducation', 'deep learning\nprojects']
📌 JD Keywords: ['data preprocessing', '- strong skills', 'chatbot projects']


In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def compute_similarity(text1, text2):
    embeddings = model.encode([text1, text2])
    score = util.cos_sim(embeddings[0], embeddings[1])
    return round(score.item() * 100, 2)

match_score = compute_similarity(resume_text, jd_text)
print("🔢 Match Score:", match_score, "%")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔢 Match Score: 69.31 %


In [7]:
missing_skills = list(set(jd_keywords) - set(resume_keywords))
print("❌ Missing Skills from Resume:", missing_skills)


❌ Missing Skills from Resume: ['data preprocessing', 'chatbot projects', '- strong skills']
